In [202]:
import torch
import torch.nn as nn


class DeepGRUModel(nn.Module):
    def __init__(self, vocab_size:int, embedding_dim:int):
        super(DeepGRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)  
        self.gru = nn.GRU(embedding_dim, 128, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
        self.fc1 = nn.Linear(128 * 2, 64) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)  

        
    def forward(self, x):
        x = self.embedding(x)  
        x, _ = self.gru(x)
        x = torch.mean(x, dim=1)  
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x) 
        return x  

In [203]:
model = DeepGRUModel(195, 128)

In [204]:
model.load_state_dict(torch.load("model.pth", weights_only=False))
model.eval()



DeepGRUModel(
  (embedding): Embedding(195, 128, padding_idx=0)
  (gru): GRU(128, 128, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [205]:
import pandas as pd
import torch

df = pd.read_csv("password_dataset.csv")

passwords = df["password"].astype(str).tolist()
y = df["is_meaningful"].values

unique_chars = sorted(set("".join(passwords)))
char_to_idx = {char: i + 1 for i, char in enumerate(unique_chars)}

max_len = max(len(p) for p in passwords)


def predict_password(password:str, device:str="cpu") -> float:
    encoded_pw = encode_password(password)
    
    input_tensor = encoded_pw.unsqueeze(0).to(device) 
    model.eval() 
    with torch.no_grad():
        output = model(input_tensor).squeeze()  
        prob = torch.sigmoid(output)  

    return prob.item()  


def encode_password(password:str) -> torch.Tensor:
    encoded = [char_to_idx.get(char, 0) for char in password]
    padded = encoded + [0] * (max_len - len(encoded))
    return torch.tensor(padded[:max_len], dtype=torch.long)



In [206]:
import time
from collections import deque
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class BruteForceDetector:
    def __init__(self, threshold=10, time_window=10, similarity_threshold=0.7, rate_limit=0.5, is_meaningful_treshold=0.5):
        self.threshold = threshold
        self.time_window = time_window  
        self.similarity_threshold = similarity_threshold  
        self.rate_limit = rate_limit
        self.is_meaningful_treshold = is_meaningful_treshold
        self.attempts = deque()
        self.vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 3))

    def detect(self, password):
        current_time = time.time()

        self.clean_old_attempts(current_time)
        
        if self.is_too_fast():
            print("[ALERT] Rapid brute-force attack detected! Requests are too frequent.")
            return True
        
        if self.is_similar_attempt(password):
            return False  
        
        self.attempts.append((current_time, password))

        if len(self.attempts) >= self.threshold:
            print("[ALERT] Brute-force attack detected! Total attempts:", len(self.attempts))
            return True
        
        return False
    
    def clean_old_attempts(self, current_time):
        while self.attempts and self.attempts[0][0] < current_time - self.time_window:
            self.attempts.popleft()
    
    def is_similar_attempt(self, new_password):
        passwords = [p for _, p in self.attempts]
        if not passwords:
            return False
        
        vectors = self.vectorizer.fit_transform(passwords + [new_password])
        similarity_matrix = cosine_similarity(vectors)
        similarities = similarity_matrix[-1][:-1]  
        
        return any(sim >= self.similarity_threshold for sim in similarities)
    
    def is_too_fast(self):
        if len(self.attempts) < 3:
            return False
        
        timestamps = [t for t, _ in self.attempts][-3:] 
        avg_interval = sum(timestamps[i] - timestamps[i-1] for i in range(1, len(timestamps))) / (len(timestamps) - 1)

        return avg_interval < self.rate_limit


# Case 1: Real Brute Force Attack
Here, an attacker tries 20 completely different passwords in a short period.

In [207]:
import time

brute_force_detector = BruteForceDetector()

brute_force_attempts = [
    "password123", "letmein", "qwerty", "trustno1", "sunshine",
    "iloveyou", "admin123", "football", "welcome", "dragon",
    "123456789", "superman", "shadow", "baseball", "michael",
    "ninja", "princess", "starwars", "harley", "batman"
]

for pwd in brute_force_attempts:
    if brute_force_detector.detect(pwd):
        break
    time.sleep(1)


[ALERT] Brute-force attack detected! Total attempts: 10


# Case 2: Random Nonsense Passwords
Attacker is trying random passwords too fast.

In [208]:
brute_force_detector = BruteForceDetector()

random_attempts = [
    "ajf?38r2n!", "x92#u@q", "8*skldja", "&-h#q9t3m", "!@*!'4d1-2",
    "j2js@!1kl", "?'*49wsad", "!?'0*asv'", "8?8q4l2m1", "%&19skd",
    "x&y!9z2@", "7a8?0_b9c#1", "m$%n#r^t", "a?!^2cd!@#$", "z*09c3v",
    "kdl*-w!@?!", "m,n.'b!vcxz!", "q'!dı$2?", "lkjh=gf*-dsa!", "qz*ws2xe'!dc"
]


for pwd in random_attempts:
    if brute_force_detector.detect(pwd):
        break


[ALERT] Rapid brute-force attack detected! Requests are too frequent.


# Case 3: Mixed Brute Force Attack
Here, the attacker tries both similar and different passwords.

In [209]:
import time

brute_force_detector = BruteForceDetector()
mixed_attempts = [
    "password123", "password124", "adminpass", "qwerty12",
    "welcome!", "football123", "iloveyou", "superadmin",
    "hacked123", "mypassword", "dragon123", "trustno2",
    "letmein1", "baseball99", "shadow12", "sunshine777",
    "harley21", "michael456", "starwarsX", "batman2000"
]

for pwd in mixed_attempts:
    if brute_force_detector.detect(pwd):
        break
    time.sleep(1)


[ALERT] Brute-force attack detected! Total attempts: 10


# Case 4: Forgotten Password Scenario
A user forgets their password and tries small variations of it.


In [210]:
import time

brute_force_detector = BruteForceDetector()
similar_passwords = [
    "password123", "password124", "password125", "password126",
    "password127", "password128", "password129", "password130",
    "password131", "password132", "password133", "password134",
    "password135", "password136", "password137", "password138",
    "password139", "password140", "password141", "password142"
]


for pwd in similar_passwords:
    if brute_force_detector.detect(pwd):
        break
    time.sleep(1)
